<a href="https://colab.research.google.com/github/mehrn79/text_classification/blob/train_model/train_model_%26_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import nltk
import numpy as np
import pandas as pd
from collections import OrderedDict
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import collections
import re
from collections import Counter, defaultdict
from oauth2client.client import GoogleCredentials
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from keras.layers import Dense, Input, LSTM, Embedding, Dropout
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Convolution1D
from keras.layers import CuDNNLSTM
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.models import Model
from keras import optimizers
import numpy as np
import pandas as pd
from keras.utils.np_utils import to_categorical
from keras.metrics import categorical_accuracy
from keras.utils.vis_utils import plot_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models import KeyedVectors
import codecs
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
! gdown --id 1wbZQm1FimgMmRDUXCnLa1wBRz7DnJZJd
! gdown --id 1vaX5SO_wGoiucO9P-bF--J7WNj-qUJhx

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1wbZQm1FimgMmRDUXCnLa1wBRz7DnJZJd
To: /content/final_epoch.h5
100% 834M/834M [00:04<00:00, 177MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1vaX5SO_wGoiucO9P-bF--J7WNj-qUJhx
To: /content/chart.csv
100% 974M/974M [00:05<00:00, 183MB/s]


In [3]:
with open('/content/chart.csv') as f:
  chart = f.read()

In [4]:
chart = pd.read_csv('chart.csv')
text = chart['clean_text'].values
Categories= chart['Cat'].values

In [5]:
category_le = LabelEncoder()
Categories = category_le.fit_transform(Categories)

category_x = text

category_y = Categories


In [6]:
x_cat_train, x_cat_test, y_cat_train, y_cat_test = train_test_split(category_x, category_y, test_size=0.20)
print('--Category Predictor--')
print('Train size: ',len(x_cat_train))
print('Test size : ',len(x_cat_test))

--Category Predictor--
Train size:  132180
Test size :  33045


In [7]:
cat_vectorizer = TfidfVectorizer(lowercase=False, ngram_range=(1,1), min_df=0.005)
# train set
x_cat_train_corpus = x_cat_train
# test set
x_cat_test_corpus = x_cat_test

# Fit vectorizer
cat_vectorizer.fit(x_cat_train_corpus)

# Print vocabulary
cat_vocab = cat_vectorizer.get_feature_names()
print(cat_vocab)
print('Vocab length:', len(cat_vocab))

# Transform vectorizer over train and test set
x_cat_train_vec = cat_vectorizer.transform(x_cat_train_corpus).toarray()
x_cat_test_vec = cat_vectorizer.transform(x_cat_test_corpus).toarray()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['آئين', 'آب', 'آباد', 'آبادي', 'آبهاي', 'آبي', 'آبياري', 'آت', 'آتي', 'آثار', 'آثاري', 'آحاد', 'آخرين', 'آد', 'آداب', 'آدمي', 'آذر', 'آذربايج', 'آذرماه', 'آر', 'آرا', 'آراء', 'آرام', 'آراي', 'آرزو', 'آرزوي', 'آرمانهاي', 'آرژانتين', 'آزاد', 'آزادسازي', 'آزادي', 'آزادگ', 'آزار', 'آزماي', 'آزمايشي', 'آزمايشگاه', 'آزمون', 'آس', 'آساني', 'آساي', 'آستانه', 'آسوشيتدپرس', 'آسيا', 'آسياي', 'آسيايي', 'آسيب', 'آسيبهاي', 'آسيبپذير', 'آشاميدني', 'آشتي', 'آشكار', 'آشنا', 'آشنايي', 'آغاز', 'آف', 'آفتاب', 'آفتابي', 'آفريقا', 'آفريقاي', 'آفريقايي', 'آفرين', 'آقا', 'آقاي', 'آل', 'آلاينده', 'آلماني', 'آلن', 'آلوده', 'آلودگي', 'آماده', 'آمادگي', 'آمار', 'آمارهاي', 'آماري', 'آمد', 'آمدن', 'آمدند', 'آمده', 'آمده_اس', 'آمده_اند', 'آمده_بود', 'آمريكا', 'آمريكاس', 'آمريكاي', 'آمريكايي', 'آموز', 'آموزشي', 'آموزي', 'آميز', 'آميزي', 'آنجايي', 'آنكارا', 'آنكه', 'آنهايي', 'آهن', 'آهنگ', 'آواز', 'آور', 'آورد', 'آوردن', 'آوردند', 'آورده', 'آورده_اس', 'آورده_اند', 'آورند', 'آوري', 'آوريل', 'آي', 'آيد', 'آيند', 'آينده

In [8]:
titles=[]
test_titles=[]

for data in x_cat_train :
  arr=data.split(' ')
  s=' '.join(arr[:500])
  titles.append(s)

for data in x_cat_test :
  arr=data.split(' ')
  s=' '.join(arr[:500])
  test_titles.append(s)

In [9]:
lstm_x_train = titles

# test set
lstm_x_test = test_titles


In [10]:
num_words = 10000

# create the tokenizer
tokenizer = Tokenizer(num_words=num_words)

# fit the tokenizer on the documents
tokenizer.fit_on_texts(x_cat_train_corpus)

# sequence encode
encoded_docs = tokenizer.texts_to_sequences(lstm_x_train)

# pad sequences
max_length = max([len(s.split()) for s in lstm_x_train])
x_train_padded = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

# define vocabulary size (largest integer value)
vocab_size = len(tokenizer.word_index) + 1

encoded_docs = tokenizer.texts_to_sequences(lstm_x_test)
x_test_padded = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [11]:
y_cat_train_classes = np.unique(y_cat_train)
y_cat_train_classes_len = len(y_cat_train_classes)

y_cat_test_classes = np.unique(y_cat_train)
y_cat_test_classes_len = len(y_cat_test_classes)

In [13]:
categorical_y_train = to_categorical(y_cat_train, y_cat_train_classes_len)
categorical_y_test = to_categorical(y_cat_test, y_cat_test_classes_len)

model_name = "text_classification.h5"
mcp_save = ModelCheckpoint(filepath=model_name,
                           save_best_only=True,
                           monitor='val_loss',
                           mode='min',
                           verbose=1)

model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=max_length))
model.add(Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(100, return_sequences=True, name='lstm_layer')))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.1))
model.add(Dense(100, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(y_cat_train_classes_len, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[categorical_accuracy])

model.summary()
batch_size = 8
epochs = 10
hist = model.fit(x_train_padded, categorical_y_train, batch_size=batch_size, epochs=epochs,
                           callbacks=mcp_save)

loss, acc = model.evaluate(x_test_padded, categorical_y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))
model.save("final_epoch_"+model_name)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 50)           17322550  
                                                                 
 bidirectional_1 (Bidirectio  (None, 500, 200)         121600    
 nal)                                                            
                                                                 
 global_max_pooling1d_1 (Glo  (None, 200)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_2 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               20100     
                                                                 
 dropout_3 (Dropout)         (None, 100)              

In [14]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r "/content/final_epoch_text_classification.h5" "/content/drive/MyDrive"

Mounted at /content/drive


In [15]:
pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 28.7 MB/s 
     |████████████████████████████████| 1.4 MB 55.7 MB/s 
     |████████████████████████████████| 233 kB 56.5 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394486 sha256=b8d6291fe736df82448c31f36886d28b67750cd54cbeca25dfd743aa57ed385c
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=155249 sha256=a56e85f548abd608f0930c2af30e48a7b104e779fec50f4d1822be9a48167120
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7


In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from hazm import *
import string
from string import digits
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 

In [17]:
with open('/content/PersianStopWords.txt') as file:
  stopLines = file.readlines()
  stopWord = [item.replace('\n',"") for item in stopLines]

In [18]:
Categories_type= chart['Cat'].values

In [19]:
def remove_digits(sent):
  remove_digits = str.maketrans('', '', digits)
  sentance =' '.join([w.translate(remove_digits) for w in sent.split(' ')]) 
  return sentance


In [20]:
def cleaning(sentance) :
  words= []
  cleanSent=[]
  stemmer = Stemmer()

  words=word_tokenize(sentance)

  for wordSent in words :
    if wordSent not in stopWord :
      stermmering=stemmer.stem(wordSent)
      cleanSent.append(stermmering)
  
  sent =' '.join(map(str, cleanSent))

  return remove_digits(sent)

In [21]:
def predict_cat(texts):
  cats=[]
  clean_text=[]
  num_words = 10000
  # tokenizer = Tokenizer(num_words=num_words)
  # tokenizer.fit_on_texts(x_cat_train)

  for x in texts:
    clean_text.append(cleaning(x))
  
  for text in clean_text :
 
    text2sent = tokenizer.texts_to_sequences([text])
    
    max_length = max([len(s.split()) for s in text])
    pad_text2sent = pad_sequences(text2sent, maxlen=max_length)
    
    cat = model.predict([pad_text2sent])[0]
    cat_index = np.where(cat == max(cat))
    
    cats.append(Categories_type[cat_index])
  return cats

In [22]:
text = chart['text'].values

In [23]:
text=text[:100]

In [26]:
test=[]
for t in text :
  arr=t.split(' ')
  s=' '.join(arr[:400])
  test.append(s)

In [27]:
arr=predict_cat(test)

In [28]:
arr

[array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['shahr'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['shahr'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['havad'], dtype=object),
 array(['h